<a href="https://colab.research.google.com/github/fininsight/page-election/blob/master/mhlee/%EC%9D%B4%EB%AF%BC%ED%98%B8_%EB%89%B4%EC%8A%A4%EB%B8%94%EB%A1%9C%EA%B7%B8_%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 기본 변수

In [136]:
files = ["./data/21대총선_news.json"]
re_char = "[^ \u3131-\u3163\uac00-\ud7a3a-zA-Z0-9.,'/%]+"

# 분석 대상파일 읽기

In [137]:
import pandas as pd

df = pd.DataFrame()
for file in files : #파일명 목록에 있는 파일들 읽어서 결합
    df = df.append(pd.read_json(file)[['url','date','title','content']], sort=False)
#df["date"]= pd.to_datetime(df["date"]) 
df.shape

(1731, 4)

## 일자별 빈도수 계산

In [74]:
import matplotlib as mpl
df_count = df.groupby([df['date'].dt.to_period('d')])['date'].agg(['count'])
df_count.unstack().plot.line()

AttributeError: Can only use .dt accessor with datetimelike values

In [161]:
import re
from konlpy.tag import Mecab
mecab = Mecab()
cleasing_re = re.compile(re_char)
news_list = [news for news in df["content"].tolist() if len(news) > 100]
    
tokenized_doc = []
cnt = 0

for news in news_list :
    NP = ""
    doc = []
    text = cleasing_re.sub('', news).replace(" ","")
    #tokenized_doc.append([ w for w in mecab.nouns(text) if len(w)>1 ])
    
    #print(cleasing_re.sub('', row))
    #print(mecab.pos(text))
    #print(tokenized_doc)
    #break
    for pos in mecab.pos(text) :
        if (pos[1][0] == "N") :
            NP += pos[0] + " "
        elif pos[1][0] != "N" :
            if len(NP) > 2  :
                doc.append(NP.strip())
            NP = ""

            if (pos[1][0] == "V") & (len(pos[0]) > 1) :
                doc.append(pos[0])
                
                
    tokenized_doc.append(doc)
    cnt += 1
    
    if cnt % 1000 == 0 :
        print(cnt)

1000


In [162]:
print(cleasing_re.sub('', news_list[1]).replace(" ",""))
print(tokenized_doc[1])

'더불어민주당이해찬대표가1일오전여의도당사에서열린더불어민주당신년인사회에서새해인사말을하고있다./연합뉴스googletag.cmd.pushfunctiongoogletag.display'divgptad15589479793640'자유한국당김병준비상대책위원장과박관용상임고문,의원,당직자들이1일오전서울영등포구당사에서열린자유한국당신년인사회에서떡케이크를자르고있다./연합뉴스더불어민주당,자유한국당등여야는1일21대총선과차기대선승리를결의하며기해년첫날을시작했다.민주당은1일1일집권3년차결의를다지며1년여앞으로다가온21대총선의압도적승리,나아가2022년대선에서의정권재창출의지도과감하게내보였다.평화와경제를전면에내세워국정운영의주도권을다잡고문재인정부의개혁드라이브에힘을더하겠다는각오다.민주당은여당으로서2년째맞는새해첫날오전여의도당사에서신년인사회를겸한단배식을개최했다.단배식에는이해찬대표와홍영표원내대표,박주민박광온설훈김해영이수진최고위원,김원기정세균전국회의장등당지도부와주요인사들이대거참석했고박원순서울시장도지난해에이어자리했다.참석자들은행사장한가운데마련된2019평화와경제새로운100년이라고적힌대형시루떡과떡국을나눠먹으며화합을약속했다.이해찬대표는인사말에서지난해에는남북간70년분단체제가평화공존체제로전환되는역사적획을그었다고평가했다.특히내년총선압승과2022년정권재창출로이나라평화와민주주의를지키는마지막보루의역할을다하자고강조했다.이대표는이어경제라는게금방좋아질수는없지만,꾸준히노력하면튼튼한기반을통해잘끌어나갈수있다며민생경제를활성화하는데당은최선을다하겠다고덧붙였다.홍영표원내대표는앞으로도많은시련과도전이기다리고있다며맞바람을향해돛을펼친다는역풍장범이라는말이있다.이해찬대표를중심으로더욱단결하는한해가되도록하자고말했다.googletag.cmd.pushfunctiongoogletag.display'divgptad15670434594650'선진규전국노인위원장은전당원이하나로뭉쳐어떤폭풍도헤쳐나가는자세가중요하다며이힘으로문재인대통령의국정수행을뒷받침하자고했다.20대국회전반기국회의장을지낸정세균의원은집권여당의높은품격과책임의식으로국민을잘섬기는한해가되도록하자고했고,박원순서울시장은

In [163]:
import gensim
from gensim import corpora
from gensim import models
dictionary = corpora.Dictionary(tokenized_doc)
corpus = [dictionary.doc2bow(text) for text in tokenized_doc]
print(corpus[1]) # 수행된 결과에서 두번째 뉴스 출력. 첫번째 문서의 인덱스는 0

from gensim.models import TfidfModel

tfidf_model = TfidfModel(corpus)
corpus_tfidf = tfidf_model[corpus]

import gensim
NUM_TOPICS = 10 #20개의 토픽, k=20
ldamodel = gensim.models.ldamodel.LdaModel(corpus_tfidf, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
#ldamodel = models.LdaMulticore(corpus, id2word=dictionary, num_topics=NUM_TOPICS, workers=8)

[(1, 2), (40, 1), (44, 2), (46, 1), (47, 1), (48, 3), (60, 1), (68, 1), (71, 3), (100, 1), (109, 2), (114, 4), (119, 1), (121, 1), (134, 1), (158, 1), (164, 1), (172, 2), (186, 1), (194, 2), (202, 1), (214, 1), (215, 2), (216, 1), (217, 1), (218, 1), (219, 1), (220, 2), (221, 1), (222, 4), (223, 4), (224, 1), (225, 1), (226, 1), (227, 1), (228, 1), (229, 1), (230, 1), (231, 1), (232, 1), (233, 1), (234, 1), (235, 1), (236, 1), (237, 1), (238, 1), (239, 1), (240, 1), (241, 1), (242, 2), (243, 1), (244, 1), (245, 1), (246, 1), (247, 1), (248, 1), (249, 1), (250, 1), (251, 1), (252, 1), (253, 1), (254, 1), (255, 1), (256, 1), (257, 1), (258, 1), (259, 1), (260, 1), (261, 1), (262, 2), (263, 1), (264, 2), (265, 1), (266, 1), (267, 1), (268, 1), (269, 1), (270, 2), (271, 1), (272, 2), (273, 1), (274, 1), (275, 1), (276, 1), (277, 1), (278, 1), (279, 1), (280, 1), (281, 1), (282, 1), (283, 1), (284, 1), (285, 1), (286, 1), (287, 1), (288, 1), (289, 1), (290, 1), (291, 1), (292, 1), (293, 1),

In [164]:
topics = ldamodel.print_topics(num_words=20)
for topic in topics:
    print(topic)
    print("\n")

(0, '0.000*"임 실장" + 0.000*"단지" + 0.000*"정후" + 0.000*"유 장관" + 0.000*"디지털 포렌식" + 0.000*"사이버 선거 범죄" + 0.000*"윤 실장" + 0.000*"허위사실 공표 등 중대 선거 범죄" + 0.000*"일 서울 종로구 서울시 선거관리위원회 사이버 공정 선거 지원 단실" + 0.000*"사이버 선거 범죄 예방" + 0.000*"선거 관련 게시물 등" + 0.000*"데이터 분석 등 전문 요원" + 0.000*"응할" + 0.000*"정기" + 0.000*"남씨" + 0.000*"엄정대" + 0.000*"집중 모니터링" + 0.000*"이전 의원" + 0.000*"윤종필 의원" + 0.000*"단속 활동"')


(1, '0.000*"김전 대변인" + 0.000*"겸전 청와대 대변인" + 0.000*"예비후보자" + 0.000*"함수 관계" + 0.000*"노동 회의소" + 0.000*"박 실장" + 0.000*"우전 대사" + 0.000*"특별 당비" + 0.000*"하승수" + 0.000*"경총" + 0.000*"등록 금심" + 0.000*"특수 공무집행 방해" + 0.000*"매각" + 0.000*"아시아 경제 임춘한 기자" + 0.000*"이종훈" + 0.000*"전과" + 0.000*"김은빈 기자" + 0.000*"노사 관계" + 0.000*"출판기념회" + 0.000*"나 의원"')


(2, '0.000*"김현미 국토부 장관" + 0.000*"유은혜 교육부 장관 왼쪽" + 0.000*"구로" + 0.000*"민주당 출신 현직 장관" + 0.000*"김 차장" + 0.000*"출판기념회" + 0.000*"남선 알미늄" + 0.000*"박전 대장" + 0.000*"년 위원회" + 0.000*"대 선거 범죄" + 0.000*"오신환 의원" + 0.000*"유구 청장" + 0.000*"김 비서관" + 0.000*"숙의" + 0.000*"조수석" + 0.000*"구민" + 0.000*"민주당 구

In [165]:
from gensim.summarization import keywords
from gensim.summarization.summarizer import summarize
import re
cleasing_re = re.compile(re_char)

for i in range(len(corpus)) :
    
    def getKey(item):
        return item[1]

    topic = sorted(ldamodel[corpus[i]], key=getKey, reverse=True)[0]
    #print(topic)
    if (topic[0] == 0) : #& (topic[1] > 0.6) :
        try :
            print(i)
            print(news_list[i])
            print(ldamodel[corpus[i]])
            print(keywords(' '.join(tokenized_doc[i])).split('\n'), 0.2)
            print(summarize(hangul.sub('', news_list[i]), 0.1))
            print("\n")
        except :
            pass



864
['‘재외동포 투표참여운동 워싱턴본부’ 발대식이 지난 17일 애난데일 중화원에서 열렸다. 내년 4월 실시되는 21대 총선을 앞두고 재외선거 투표율 증진, 재외국민 권익 신장에 뜻을 같이하는 한인단체들이 함께 했다. 워싱턴 본부장에는 이기창(사람사는세상-워싱턴 전 대표, 뒷줄 오른쪽 두번째), 부본부장에는 홍미정(사사세 대표)씨가 선임됐다. 이 본부장은 “재외동포 투표참여운동은 시민단체들의 자발적 참여로 시작됐다”며 “특정정당이나 특정 정치인을 지지하는 선거운동이 아닌 재외국민유권자들의 투표 참여율을 높이기 위한 유권자 등록운동”이라고 밝혔다. 이날 발대식은 사사세, 더불어민주당 워싱턴 대의원협의회, 민주동우회에서 후원했다. &lt;유제원기자&gt; ']
[(0, 0.5333748), (5, 0.44784763)]
['워싱턴'] 0.2





In [155]:
news_list[22]

'[]'

In [112]:
from gensim.summarization import keywords
from gensim.summarization.summarizer import summarize
import re
cleasing_re = re.compile(re_char)



for i in range(10) :
    #print(hangul.sub('', news_list[i]))
    #print(tokenized_doc[i])
    print(ldamodel[corpus[i]])
    print(keywords(' '.join(tokenized_doc[i])).split('\n'), 0.2)
    print(summarize(hangul.sub('', news_list[i]), 0.1))
    print("\n")


[(0, 0.06707124), (1, 0.14864512), (2, 0.020175269), (13, 0.07625538), (17, 0.68364424)]
['민주당', '빚어진', '정규직', '홍준표', '운전자 황교안', '대통령'] 0.2
손석희  대표이사 사장이 직원들에게 걱정시켜 미안하다라며 앞으로 흔들림 없이 헤쳐나가겠다라고 전했다.
나도 황당하고 당혹스러운 게 사실이라면서 이같이 말했다.
충청인들은 내일이 국회의원 선거일이라면 어느 당을 찍겠느냐라는 물음에 민주당 전국 자유한국당 전국 바른미래당 전국 정의당 전국 민주평화당 전국 무당층 전국 로 집계됐다.
한국갤럽은 이를현재 정당 지지도와 총선 투표 의향 정당을 비교하면 더불어민주당   자유한국당  정의당  바른미래당  민주평화당  등 주요 정당들 모두 변화 폭이  이내로 미미했다.고 말해 정당 지지도 투표의항에 비례함을 보여줬다.
갤럽은 현재 지지하는 정당이 없는 무당 층은 총선 투표 의향 정당으로 더불어민주당  자유한국당  바른미래당  정의당  민주평화당 를 선택해 진보 정당과 중도보수 정당 합산 비율이 각각  로 큰 차이 없었다.라고 분석했다.


[(0, 0.015287751), (6, 0.019299349), (9, 0.014183505), (13, 0.5565552), (17, 0.38948822)]
['보인다', '최고위원', '위원장'] 0.2
정치권에서는 바른정당 계열 인사들의 연이은 탈당으로 당내 입지가 좁아지는 유 의원의 마지막 퇴로를 보수대통합으로 꼽고 있지만 한국당과의 통합을 꺼려하는 손학규 대표 등 국민의당 계열 의원들의 반대로 이마저도 여의치 않아 보인다.


[(0, 0.10050921), (2, 0.08540287), (13, 0.41907308), (17, 0.38635746)]
['정무수석', '정의용', '대통령'] 0.2
임종석 비서실장을 비롯해 한병도 정무수석 윤영찬 국민소통수석 등이 거론되고 있는 가운데 임 실장의 후임에는 년 대선 때 조직본부장을 맡은

In [113]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)
pyLDAvis.display(vis)

KeyboardInterrupt: 